In [17]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [5]:
df = pd.read_csv('StockEtablissement_utf8.csv', nrows = 100000, skipinitialspace=True)
for k in df.columns:
    print(f'{k} {df[k].isna().sum()}')

FileNotFoundError: [Errno 2] No such file or directory: 'StockEtablissement_utf8.csv'

In [4]:
df.shape

(10000000, 5)

In [4]:
df.head()

,siret,codePostalEtablissement,libelleCommuneEtablissement,codeCommuneEtablissement,activitePrincipaleEtablissement
0,32517500016,98770.0,MANIHI,98727,32.12Z
1,32517500024,84140.0,AVIGNON,84007,47.89Z
2,32517500032,84000.0,AVIGNON,84007,32.12Z
3,32517500040,13420.0,GEMENOS,13042,32.12Z
4,32517500057,13004.0,MARSEILLE 4,13204,32.12Z


In [5]:
for k in df.columns:
    print(k)
    print(df[k].isna().sum())

siret
0
codePostalEtablissement
25536
libelleCommuneEtablissement
25536
codeCommuneEtablissement
25536
activitePrincipaleEtablissement
9191


## Upload du csv sur la base de données MySQL

In [3]:
#Fonction de connexion au serveur DataIA
import sqlite3
import pymysql
import yaml
from sqlalchemy import create_engine
import pandas as pd

def connexion_mysql():
    with open ('connect.yml', 'r') as f:
        conf = yaml.safe_load(f)
    my = conf['MYSQL']
    conf['MYSQL']['user']
    engineMY = create_engine(f"mysql+pymysql://{my['user']}:{my['password']}@{my['host']}:{my['port']}/bdd_menou", echo=False)
    return(engineMY)

In [8]:
#Upload de la base de données par chunk de 100 000 lignes

engine = connexion_mysql()
L = ['siret', 'numeroVoieEtablissement', 'typeVoieEtablissement', 'libelleVoieEtablissement', 'codePostalEtablissement', 'codeCommuneEtablissement', 'dateDebut', 'etatAdministratifEtablissement', 'activitePrincipaleEtablissement']
df = pd.read_csv('StockEtablissement_utf8.csv', chunksize = 100000, usecols= L)
for i, k in enumerate(df) :
    print(f'chunk {i}')
    k.to_sql('Siren', con=engine, if_exists='append', index=False)

C:\Users\kylli\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3364: DtypeWarning: Columns (20) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


chunk 0
chunk 1
chunk 2
chunk 3
chunk 4
chunk 5
chunk 6
chunk 7
chunk 8
chunk 9
chunk 10
chunk 11
chunk 12
chunk 13
chunk 14
chunk 15
chunk 16
chunk 17
chunk 18
chunk 19
chunk 20
chunk 21


KeyboardInterrupt: 

## Cartographie

In [4]:
#Creation d'une fonction renvoyant une carte avec un marqueur par entreprise française pour une catégorie donnée
import folium
import pandas as pd

def Category_Mapping(cat):
    engine = connexion_mysql()
    df_cat = pd.read_sql_query(('''SELECT  POSTE.nomCommune,
                                POSTE.latitude,
                                POSTE.longitude,
                                SIRENE.siret,
                                SIRENE.typeVoieEtablissement,
                                SIRENE.libelleVoieEtablissement,
                                NAF.description
                                FROM SIRENE
                                JOIN POSTE ON SIRENE.codeCommuneEtablissement = POSTE.codeCommune
                                JOIN NAF ON SIRENE.activitePrincipaleEtablissement = NAF.naf
                                WHERE (NAF.description = %s);'''),
                                engine,
                                params=[cat])
    
    CentreFrance = [46.3622, 1.5231]

    map = folium.Map(location = CentreFrance,
                     zoom_start = 6,
                     tiles="Stamen Terrain")
    for idx in df_cat.index:
        tooltip = ""
        folium.Marker([df_cat['longitude'][idx], df_cat['latitude'][idx]], popup="<i></i>", tooltip=tooltip).add_to(map)
    return(map)

In [92]:
#Creation d'une fonction renvoyant une carte avec une heatmap des entreprises françaises pour une catégorie donnée
import folium
from folium.plugins import HeatMap
import numpy as np
import pandas as pd
from statistics import mean

def Category_Mapping(cat):
    engine = connexion_mysql()
    df_cat = pd.read_sql_query(('''SELECT  POSTE.nomCommune,
                                POSTE.latitude,
                                POSTE.longitude,
                                SIRENE.siret,
                                SIRENE.typeVoieEtablissement,
                                SIRENE.libelleVoieEtablissement,
                                NAF.description
                                FROM SIRENE
                                JOIN POSTE ON SIRENE.codeCommuneEtablissement = POSTE.codeCommune
                                JOIN NAF ON SIRENE.activitePrincipaleEtablissement = NAF.naf
                                WHERE (NAF.description = %s);'''),
                                engine,
                                params=[cat])
    
    #Ajout des coordonnées à un tableau
    heat_data = [[row['longitude'],row['latitude']] for index, row in df_cat.iterrows()]
    
    #Calcul des coordonées moyennes pour l'activité recherchée            
    try :
        Centre = [mean([float(Rizi[k][0]) for k in range(len(Rizi))]),
                  mean([float(Rizi[k][1]) for k in range(len(Rizi))])]
    
    #Sinon, centre géographique de la France
    except:
        Centre = [46.3622,
                  1.5231]

    #Création d'un objet carte vide centré sur la zone voulue.
    map = folium.Map(location = Centre,
                     zoom_start = 6)

    #Ajout de la heatmap à l'objet carte
    HeatMap(heat_data).add_to(map)

    return(map)

In [93]:
from flask import Flask
Rizi = Category_Mapping(cat = 'Culture du riz')

Rizi._repr_html_()


'<div style="width:100%;"><div style="position:relative;width:100%;height:0;padding-bottom:60%;"><span style="color:#565656">Make this Notebook Trusted to load map: File -> Trust Notebook</span><iframe srcdoc="&lt;!DOCTYPE html&gt;\n&lt;head&gt;    \n    &lt;meta http-equiv=&quot;content-type&quot; content=&quot;text/html; charset=UTF-8&quot; /&gt;\n    \n        &lt;script&gt;\n            L_NO_TOUCH = false;\n            L_DISABLE_3D = false;\n        &lt;/script&gt;\n    \n    &lt;style&gt;html, body {width: 100%;height: 100%;margin: 0;padding: 0;}&lt;/style&gt;\n    &lt;style&gt;#map {position:absolute;top:0;bottom:0;right:0;left:0;}&lt;/style&gt;\n    &lt;script src=&quot;https://cdn.jsdelivr.net/npm/leaflet@1.6.0/dist/leaflet.js&quot;&gt;&lt;/script&gt;\n    &lt;script src=&quot;https://code.jquery.com/jquery-1.12.4.min.js&quot;&gt;&lt;/script&gt;\n    &lt;script src=&quot;https://maxcdn.bootstrapcdn.com/bootstrap/3.2.0/js/bootstrap.min.js&quot;&gt;&lt;/script&gt;\n    &lt;script

In [104]:
engine = connexion_mysql()

df_naf = pd.read_sql_query(('''SELECT  distinct(NAF.description), NAF.naf
                            FROM bdd_menou.NAF'''), engine)


df_naf['type'] = 0
df_naf

for k in df_naf.index:
    l = len(df_naf.loc[k, 'naf'])
    if l == 2:
        df_naf.loc[k, 'type'] = 'Catégorie Principale'
    elif l == 4:
        df_naf.loc[k, 'type'] = 'Catégorie Secondaire'
    elif 4 < l < 6:
        df_naf.loc[k, 'type'] = "Description de l'activité"
    elif l >= 6:
        df_naf.loc[k, 'type'] = "Nom de Section"
    else :
        df_naf.loc[k, 'type'] = "Autre"
df_naf    


,description,naf,type
0,"AGRICULTURE, SYLVICULTURE ET PÊCHE",SECTION A,Nom de Section
1,"Culture et production animale, chasse et servi...",01,Catégorie Principale
2,Cultures non permanentes,01.1,Catégorie Secondaire
3,"Culture de céréales (à l'exception du riz), de...",01.11,Description de l'activité
4,"Culture de céréales (à l'exception du riz), de...",01.11Z,Nom de Section
5,Culture du riz,01.12,Description de l'activité
6,Culture du riz,01.12Z,Nom de Section
7,"Culture de légumes, de melons, de racines et d...",01.13,Description de l'activité
8,"Culture de légumes, de melons, de racines et d...",01.13Z,Nom de Section
9,Culture de la canne à sucre,01.14,Description de l'activité
